### Импорты, параметры ColBERT

In [1]:
from interface import prepare_tsv, save_index, top_n_similar, Collection, pair_scores, calculate_cosine_similarity
import pandas as pd
import os

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/triples_X1_13_categories_use_ib_negatives/none/2024-01/26/10.49.44/checkpoints/colbert-5387-finish"
experiment = "colbert-5387"

doc_maxlen = 300
nbits = 2   # bits определяет количество битов у каждого измерения в семантическом пространстве во время индексации
nranks = 1  # nranks определяет количество GPU для использования, если они доступны
kmeans_niters = 4 # kmeans_niters указывает количество итераций k-means кластеризации; 4 — хороший и быстрый вариант по умолчанию.  

### Подготовка моделей

В данном примере готовим модели в качестве того на что матчить оффера. Данные должны быть в формате, пригодном для индексации с помощью ColBERT. В общем случае в индексе может быть и смесь моделей и уже ранее моделизованных офферов

In [2]:
pth_models = "/home/sondors/Documents/price/ColBERT_data/18_categories/test/models_18_categories.csv"
df_models = pd.read_csv(pth_models, sep=";")
df_models

,model_id,name,brand_name,full_name,category_name,category_id
0,623742,920-005619,Logitech,Logitech 920-005619,"чехлы, обложки для гаджетов (телефонов, планше...",3994
1,721952,Zipper Bag,Hama,Hama Zipper Bag,"чехлы, обложки для гаджетов (телефонов, планше...",3994
2,721970,CC-3064,Nokia,Nokia CC-3064,"чехлы, обложки для гаджетов (телефонов, планше...",3994
3,751488,CKS-X7/R,Sony,Sony CKS-X7/R,"чехлы, обложки для гаджетов (телефонов, планше...",3994
4,751989,EP-031023,Era Pro,Era Pro EP-031023,"чехлы, обложки для гаджетов (телефонов, планше...",3994
...,...,...,...,...,...,...
99610,7049424,MD-108,Mivo,Mivo MD-108,портативная акустика,3904
99611,7049425,MD-165,Mivo,Mivo MD-165,портативная акустика,3904
99612,7049426,Boost 20W,Rocket,Rocket Boost 20W,портативная акустика,3904
99613,7049427,Motion 10W,Rocket,Rocket Motion 10W,портативная акустика,3904


In [3]:
id_category = {
    3902: 'диктофоны, портативные рекордеры',
    510402: 'электронные книги',
    4302: 'автомобильные телевизоры, мониторы',
    2815: 'смарт-часы и браслеты',
    3901: 'портативные медиаплееры',
    3904: 'портативная акустика',
    2801: 'мобильные телефоны',
    3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    510401: 'планшетные компьютеры и мини-планшеты',
    2102: 'наушники, гарнитуры, наушники c микрофоном',
    3903: 'радиоприемники, радиобудильники, радиочасы',
    3907: 'магнитолы',
    280801: 'GPS-навигаторы'
    }

dst_fld = "/home/sondors/Documents/price/ColBERT/tutorial"
for cat_id in id_category.keys():
    category_models = df_models[df_models.category_id == cat_id].reset_index(drop=True)
    prepare_tsv(category_models, dst_fld, cat_id)

### Индексируем модели и сохраняем индекс на диск

В id_category всего 13 категорий, в сумме 41037 шт моделей:

- ColBERT bert-base-multilingual-cased_dim_768: индексация за 760 сек на CPU с 16 ядрами без GPU, вес индекса 109 Мб

- ColBERTv2.0_dim_128: индексация за 790 сек на CPU с 16 ядрами без GPU, вес индекса 20.6 Мб

In [6]:
for cat_id in id_category.keys():
    models_colbert = Collection(path=os.path.join(dst_fld, "tsv", f"{cat_id}_models.tsv"))
    index_name = f'models_{cat_id}_{nbits}bits'
    indexer = save_index(ckpt_pth, doc_maxlen, nbits, kmeans_niters, nranks, dst_fld, experiment, models_colbert, index_name)

[Feb 11, 10:05:22] #> Loading collection...
0M 


[Feb 11, 10:05:22] #> Creating directory /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_510401_8bits 


#> Starting...
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "nbits": 8,
    "kmeans_niters": 4,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 0.0001,
    "maxsteps": 5387,
    "save_every": null,
    "warmup": 0,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 300,
    "mask_punct

/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
  0%|          | 0/50 [00:00<?, ?it/s]/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Feb 11, 10:05:26] [0] 		 # of sampled PIDs = 3839 	 sampled_pids[:3] = [1706, 3001, 41]
[Feb 11, 10:05:26] [0] 		 #> Encoding 3839 passages..


100%|██████████| 10/10 [00:06<00:00,  1.57it/s]
WARNING clustering 37169 points to 2048 centroids: please provide at least 79872 training points


[Feb 11, 10:05:59] [0] 		 avg_doclen_est = 10.191455841064453 	 len(local_sample) = 3,839
[Feb 11, 10:05:59] [0] 		 Creaing 2,048 partitions.
[Feb 11, 10:05:59] [0] 		 *Estimated* 39,124 embeddings.
[Feb 11, 10:05:59] [0] 		 #> Saving the indexing plan to /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_510401_8bits/plan.json ..
Clustering 37169 points in 128D to 2048 clusters, redo 1 times, 4 iterations
  Preprocessing in 0.00 s
  Iteration 3 (0.17 s, search 0.17 s): objective=6985.24 imbalance=1.516 nsplit=0       
[0.03, 0.032, 0.03, 0.03, 0.03, 0.031, 0.03, 0.03, 0.026, 0.031, 0.03, 0.034, 0.029, 0.035, 0.028, 0.033, 0.029, 0.029, 0.029, 0.029, 0.03, 0.031, 0.029, 0.029, 0.029, 0.029, 0.031, 0.034, 0.033, 0.031, 0.029, 0.028, 0.031, 0.028, 0.032, 0.033, 0.028, 0.029, 0.035, 0.031, 0.029, 0.031, 0.033, 0.03, 0.029, 0.03, 0.03, 0.028, 0.031, 0.031, 0.029, 0.031, 0.033, 0.027, 0.03, 0.029, 0.031, 0.028, 0.032, 0.031, 0.028, 0.029, 0.035, 0.037, 0.03, 0.029, 0

0it [00:00, ?it/s]
100%|██████████| 50/50 [00:25<00:00,  1.94it/s]

100%|██████████| 10/10 [00:06<00:00,  1.59it/s]


[Feb 11, 10:06:32] [0] 		 #> Saving chunk 0: 	 3,839 passages and 39,125 embeddings. From #0 onward.


1it [00:32, 32.40s/it]
100%|██████████| 2048/2048 [00:00<00:00, 97647.29it/s]


[Feb 11, 10:06:32] [0] 		 #> Checking all files were saved...
[Feb 11, 10:06:32] [0] 		 Found all files!
[Feb 11, 10:06:32] [0] 		 #> Building IVF...
[Feb 11, 10:06:32] [0] 		 #> Loading codes...
[Feb 11, 10:06:32] [0] 		 Sorting codes...
[Feb 11, 10:06:32] [0] 		 Getting unique codes...
[Feb 11, 10:06:32] #> Optimizing IVF to store map from centroids to list of pids..
[Feb 11, 10:06:32] #> Building the emb2pid mapping..
[Feb 11, 10:06:32] len(emb2pid) = 39125
[Feb 11, 10:06:32] #> Saved optimized IVF to /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_510401_8bits/ivf.pid.pt
[Feb 11, 10:06:32] [0] 		 #> Saving the indexing metadata to /home/sondors/Documents/price/ColBERT/tutorial/colbert-5387/indexes/models_510401_8bits/metadata.json ..
#> Joined...


### Поиск матча по индексу

Запуск 77к офферов на поиск матча среди 13 категорий или 41к шт моделей занимает 20 мин на CPU

In [7]:
offers = {
    0: 'Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)',
    1: 'Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)',
    2: 'Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)'
    }

src_fld = "/home/sondors/Documents/price/ColBERT/tutorial"
cat_id = 510401 # планшеты
index_name = f'models_{cat_id}_{nbits}bits'
models_id_colbert = Collection(path=os.path.join(src_fld, "tsv", f"{cat_id}_models_id.tsv"))

top_n = top_n_similar(offers, src_fld, nranks, experiment, index_name, models_id_colbert, 5)

[Feb 11, 10:06:45] #> Loading collection...
0M 
[Feb 11, 10:06:46] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Feb 11, 10:06:46] #> Loading codec...
[Feb 11, 10:06:46] #> Loading IVF...
[Feb 11, 10:06:46] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


[Feb 11, 10:06:46] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 2102.41it/s]

[Feb 11, 10:06:46] #> Loading codes and residuals...


...

[Feb 11, 10:06:46] Loading filter_pids_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


[Feb 11, 10:06:46] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')



#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global), 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1, 19102,  1194, 29436, 28995, 29753, 15290, 22919, 19102,
         9088, 21628,  1055,  2620,  1010,  1022,  1183, 29740,  1013, 11899,
         1183, 29740,  1010, 15536,  1011, 10882,  1009, 12562,  1010,  1196,
        14150,   102])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])



3it [00:00, 333.45it/s]


### Получаем выдачу топ N моделей для каждого оффера

In [8]:
for i in range(len(top_n)):
    print(offers[i])
    for j in range(len(top_n[i]['model_ids'])):
        id = top_n[i]['model_ids'][j]
        sim = top_n[i]['similarity'][j]
        model = list(df_models[df_models.model_id == id]['full_name'])[0]
        print(f"\t{id}: {model} --> {round(float(sim), 2)}")
    print("_"*60)

Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S8 --> 27.21
	5144477: Samsung Galaxy Tab S8+ --> 21.73
	5144479: Samsung Galaxy Tab S8 Ultra --> 21.04
	3843929: Samsung Galaxy Tab S4 10.5 SM-T835 --> 20.51
	1464907: Samsung Galaxy Tab S3 9.7 SM-T820 --> 20.06
____________________________________________________________
Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)
	5144478: Samsung Galaxy Tab S8 --> 27.29
	3843929: Samsung Galaxy Tab S4 10.5 SM-T835 --> 20.56
	623213: Samsung Galaxy Tab S 10.5 SM-T800 --> 20.5
	5144477: Samsung Galaxy Tab S8+ --> 20.46
	5144479: Samsung Galaxy Tab S8 Ultra --> 20.31
____________________________________________________________
Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)
	5144477: Samsung Galaxy Tab S8+ --> 27.56
	5144478: Samsung Galaxy Tab S8 --> 20.04
	623213: Samsung Galaxy Tab S 10.5 SM-T800 --> 19.83
	4509798: Samsung Galaxy Tab S7+ 12.4 1

In [ ]:
"""
Для эксперимента bert-base-multilingual-cased-2998:

Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S8 --> 28.97
	5144477: Samsung Galaxy Tab S8+ --> 24.97
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 24.0
	5144479: Samsung Galaxy Tab S8 Ultra --> 23.98
	816746: Samsung Galaxy Tab S2 9.7 SM-T810 --> 23.64
____________________________________________________________
Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)
	5144478: Samsung Galaxy Tab S8 --> 28.15
	5144477: Samsung Galaxy Tab S8+ --> 23.94
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 23.21
	623216: Samsung Galaxy Tab S 8.4 SM-T705 --> 22.82
	816746: Samsung Galaxy Tab S2 9.7 SM-T810 --> 22.65
____________________________________________________________
Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)
	5144477: Samsung Galaxy Tab S8+ --> 26.03
	5144478: Samsung Galaxy Tab S8 --> 20.32
	4509798: Samsung Galaxy Tab S7+ 12.4 128Gb --> 18.63
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 18.35
	5144479: Samsung Galaxy Tab S8 Ultra --> 18.31
____________________________________________________________
"""

In [ ]:
top_n

[{'model_ids': [5144478, 5144479, 3843929, 4892891, 623215],
  'similarity': [25.507810592651367,
   22.290096282958984,
   21.059688568115234,
   19.644437789916992,
   19.64250946044922]},
 {'model_ids': [5144478, 5144479, 3843929, 1464907, 623215],
  'similarity': [25.12063980102539,
   21.704925537109375,
   21.299270629882812,
   19.382617950439453,
   19.33898162841797]},
 {'model_ids': [5144477, 4509798, 5144478, 623213, 5144479],
  'similarity': [23.25652503967285,
   19.523971557617188,
   19.506370544433594,
   19.483131408691406,
   18.18914794921875]}]

### Получение similarity без индексирования

Так же можно использовать ColBERT без предварительного индексирования и поиска матча по индексу. Возможно получение similarity для пары строк, для одной строки со списком строк или для каждой пары элементов из двух списков строк. 

In [46]:
query1 = "Смартфон Apple iPhone 11 Pro Max 256Gb Gold (MGDE3RU/A)"
document1 = "Смартфон Apple iPhone 12 Pro Max 256Gb Gold"
document2 = "Apple iPhone Pro Max 11"
document3 = "модель iPhone номер 11 серия Pro Max бренд Apple"

scores = pair_scores(ckpt_pth, doc_maxlen, nbits, kmeans_niters, [query1], [document1, document2, document3], 100, "cpu")
print(f"{query1}:")
print(f"similarity = {scores[0][0]}: {document1}")
print(f"similarity = {scores[0][1]}: {document2}")
print(f"similarity = {scores[0][2]}: {document3}")


#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Смартфон Apple iPhone 11 Pro Max 256Gb Gold (MGDE3RU/A), 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  1196, 29745, 10260, 16856, 22919, 29749, 14150, 18947,
         6207, 18059,  2340,  4013,  4098, 17273, 18259,  2751,  1006, 11460,
         3207,  2509,  6820,  1013,  1037,  1007,   102,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0])

batch: 1/1
Смартфон Apple iPhone 11 Pro Max 256Gb Gold (MGDE3RU/A):
similarity = 23.38526153564453: Смартфон Apple iPhone 12 Pro Max 256Gb Gold
similarity = 27.34119415283203: Apple iPhone Pro Max 11
similarity = 23.714160919189453: модель iPhone номер 11 серия Pro Max бренд Apple


### Получение cosine similarity
Получение косинусной близости без использования преимуществ ColBERT'а, так же как ранее использовался обычный BERT. Возможно получение cosine similarity для пары строк, для одной строки со списком строк или для каждой пары элементов из двух списков строк. 

In [4]:
query1 = "Смартфон Apple iPhone 11 Pro Max 256Gb Gold (MGDE3RU/A)"
document1 = "Смартфон Apple iPhone 12 Pro Max 256Gb Gold"
document2 = "Apple iPhone Pro Max 11"
document3 = "модель iPhone номер 11 серия Pro Max бренд Apple"

cosine_sims = calculate_cosine_similarity(ckpt_pth, doc_maxlen, nbits, kmeans_niters, [query1], [document1, document2, document3], 100, "cpu")
print(f"{query1}:")
print(f"cosine_sim = {cosine_sims[0][0]}: {document1}")
print(f"cosine_sim = {cosine_sims[0][1]}: {document2}")
print(f"cosine_sim = {cosine_sims[0][2]}: {document3}")

/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')



#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Смартфон Apple iPhone 11 Pro Max 256Gb Gold (MGDE3RU/A), 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  1196, 29745, 10260, 16856, 22919, 29749, 14150, 18947,
         6207, 18059,  2340,  4013,  4098, 17273, 18259,  2751,  1006, 11460,
         3207,  2509,  6820,  1013,  1037,  1007,   102,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0])

batch: 1/1
Смартфон Apple iPhone 11 Pro Max 256Gb Gold (MGDE3RU/A):
cosine_sim = 0.6936457753181458: Смартфон Apple iPhone 12 Pro Max 256Gb Gold
cosine_sim = 0.8632790446281433: Apple iPhone Pro Max 11
cosine_sim = 0.6954565048217773: модель iPhone номер 11 серия Pro Max бренд Apple
